In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score

In [3]:

players_train_df = pd.read_csv("/content/male_players (legacy).csv") #for training
players_test_df =  pd.read_csv("/content/players_22 (1).csv") # for testing

<ipython-input-3-2f086c6ba829>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_train_df = pd.read_csv("/content/male_players (legacy).csv") #for training
<ipython-input-3-2f086c6ba829>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_test_df =  pd.read_csv("/content/players_22 (1).csv") # for testing


In [4]:
players_train_df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [5]:
players_train_df.isnull().sum()

player_id           0
player_url          0
fifa_version        0
fifa_update         0
fifa_update_date    0
                   ..
cb                  1
rcb                 1
rb                  1
gk                  1
player_face_url     1
Length: 110, dtype: int64

In [6]:
null_counts = players_train_df.isnull().sum()
pd.set_option('display.max_rows', None)
print(null_counts)

player_id                             0
player_url                            0
fifa_version                          0
fifa_update                           0
fifa_update_date                      0
short_name                            0
long_name                             0
player_positions                      0
overall                               0
potential                             0
value_eur                           890
wage_eur                            661
age                                   0
dob                                   0
height_cm                             0
weight_kg                             0
league_id                           669
league_name                         669
league_level                       1222
club_team_id                        669
club_name                           669
club_position                       669
club_jersey_number                  669
club_loaned_from                  43848
club_joined_date                   3665


In [7]:
threshold = len(players_train_df) * 0.6

# Drop columns that have more than 60% null values
players_train_df = players_train_df.dropna(thresh=threshold, axis=1)

# Display a concise summary of the resulting DataFrame
print(players_train_df.info())

# Optionally, display the first few rows to verify changes
print(players_train_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46844 entries, 0 to 46843
Columns: 101 entries, player_id to player_face_url
dtypes: float64(42), int64(16), object(43)
memory usage: 36.1+ MB
None
   player_id                                        player_url  fifa_version  \
0     158023                /player/158023/lionel-messi/150002            15   
1      20801  /player/20801/c-ronaldo-dos-santos-aveiro/150002            15   
2       9014                  /player/9014/arjen-robben/150002            15   
3      41236           /player/41236/zlatan-ibrahimovic/150002            15   
4     167495                /player/167495/manuel-neuer/150002            15   

   fifa_update fifa_update_date         short_name  \
0            2       2014-09-18           L. Messi   
1            2       2014-09-18  Cristiano Ronaldo   
2            2       2014-09-18          A. Robben   
3            2       2014-09-18     Z. Ibrahimović   
4            2       2014-09-18           M. Neuer 

In [8]:
players_train_df.head().isnull().sum()

player_id                         0
player_url                        0
fifa_version                      0
fifa_update                       0
fifa_update_date                  0
short_name                        0
long_name                         0
player_positions                  0
overall                           0
potential                         0
value_eur                         0
wage_eur                          0
age                               0
dob                               0
height_cm                         0
weight_kg                         0
league_id                         0
league_name                       0
league_level                      0
club_team_id                      0
club_name                         0
club_position                     0
club_jersey_number                0
club_joined_date                  0
club_contract_valid_until_year    0
nationality_id                    0
nationality_name                  0
preferred_foot              

In [9]:
from sklearn.preprocessing import LabelEncoder

# Assuming players_train_df is your DataFrame
# Identify non-numeric columns
non_numeric_columns = players_train_df.select_dtypes(include=['object']).columns

# Apply label encoding to non-numeric columns
label_encoders = {}
for col in non_numeric_columns:
    le = LabelEncoder()
    players_train_df[col] = le.fit_transform(players_train_df[col].astype(str))
    label_encoders[col] = le

# Display the DataFrame to ensure non-numeric columns are encoded
print(players_train_df.head())

   player_id  player_url  fifa_version  fifa_update  fifa_update_date  \
0     158023        4011            15            2                 0   
1      20801       25538            15            2                 0   
2       9014       46830            15            2                 0   
3      41236       46012            15            2                 0   
4     167495        5865            15            2                 0   

   short_name  long_name  player_positions  overall  potential  ...  cdm  rdm  \
0       12785      13444               313       93         95  ...  158  158   
1        4213       4369               820       92         92  ...  168  168   
2        1515       2194              1079       90         90  ...  176  176   
3       22431      23203              1247       90         90  ...  186  186   
4       14805      14380               533       90         90  ...   62   62   

   rwb   lb  lcb   cb  rcb   rb   gk  player_face_url  
0  152  108   83  

In [10]:
players_train_df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,4011,15,2,0,12785,13444,313,93,95,...,158,158,152,108,83,83,83,108,22,5198
1,20801,25538,15,2,0,4213,4369,820,92,92,...,168,168,162,122,115,115,115,122,26,506
2,9014,46830,15,2,0,1515,2194,1079,90,90,...,176,176,171,112,88,88,88,112,18,233
3,41236,46012,15,2,0,22431,23203,1247,90,90,...,186,186,145,118,132,132,132,118,30,633
4,167495,5865,15,2,0,14805,14380,533,90,90,...,62,62,55,54,59,59,59,54,169,7036


In [11]:
# Proceed with correlation analysis
correlation = players_train_df.corr()['overall'].sort_values(ascending=False)

# Display the correlation
print(correlation.to_frame())

# Optionally, print the top correlations
print(correlation.head(10))

                                 overall
overall                         1.000000
movement_reactions              0.804865
potential                       0.787631
wage_eur                        0.677941
passing                         0.639017
value_eur                       0.595731
lcm                             0.590911
rcm                             0.590911
cm                              0.590911
lf                              0.569377
cf                              0.569377
rf                              0.569377
lam                             0.564453
ram                             0.564453
cam                             0.564453
rm                              0.558329
lm                              0.558329
dribbling                       0.555747
lw                              0.554520
rw                              0.554520
ls                              0.544273
st                              0.544273
rs                              0.544273
rdm             

In [12]:
player_position_attributes = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
                              'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb',
                              'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

# Drop player position attributes
players_train_df = players_train_df.drop(columns=player_position_attributes)

# Display the DataFrame to ensure the columns are dropped
print(players_train_df.head())

   player_id  player_url  fifa_version  fifa_update  fifa_update_date  \
0     158023        4011            15            2                 0   
1      20801       25538            15            2                 0   
2       9014       46830            15            2                 0   
3      41236       46012            15            2                 0   
4     167495        5865            15            2                 0   

   short_name  long_name  player_positions  overall  potential  ...  \
0       12785      13444               313       93         95  ...   
1        4213       4369               820       92         92  ...   
2        1515       2194              1079       90         90  ...   
3       22431      23203              1247       90         90  ...   
4       14805      14380               533       90         90  ...   

   mentality_penalties  defending_marking_awareness  \
0                 76.0                         25.0   
1                 85.0  

In [13]:
players_train_df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,mentality_penalties,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,player_face_url
0,158023,4011,15,2,0,12785,13444,313,93,95,...,76.0,25.0,21.0,20.0,6.0,11.0,15.0,14.0,8.0,5198
1,20801,25538,15,2,0,4213,4369,820,92,92,...,85.0,22.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,506
2,9014,46830,15,2,0,1515,2194,1079,90,90,...,80.0,29.0,26.0,26.0,10.0,8.0,11.0,5.0,15.0,233
3,41236,46012,15,2,0,22431,23203,1247,90,90,...,91.0,25.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0,633
4,167495,5865,15,2,0,14805,14380,533,90,90,...,37.0,25.0,25.0,25.0,87.0,85.0,92.0,90.0,86.0,7036


In [14]:
correlation = players_train_df.corr()['overall'].sort_values(ascending=False)

# Display the correlation
print(correlation.to_frame())

# Optionally, print the top correlations
print(correlation.head(10))

                                 overall
overall                         1.000000
movement_reactions              0.804865
potential                       0.787631
wage_eur                        0.677941
passing                         0.639017
value_eur                       0.595731
dribbling                       0.555747
international_reputation        0.511026
physic                          0.471286
attacking_short_passing         0.462887
shooting                        0.456346
skill_long_passing              0.448898
mentality_vision                0.445062
skill_ball_control              0.430765
age                             0.412181
power_shot_power                0.408878
skill_curve                     0.385720
real_face                       0.379878
power_long_shots                0.378908
mentality_aggression            0.371152
attacking_crossing              0.368122
skill_fk_accuracy               0.363431
attacking_volleys               0.347683
skill_dribbling 

In [15]:
selected_features = [
    'movement_reactions',
    'potential',
    'wage_eur',
    'passing',
    'value_eur',
    'dribbling',
    'international_reputation',
    'physic',
    'shooting',
    'skill_ball_control'
]

# Select these features from the players_train_df
selected_features_df = players_train_df[selected_features]  # Including 'overall' for target variable

# Display the new DataFrame with selected features
selected_features_df.head()

,movement_reactions,potential,wage_eur,passing,value_eur,dribbling,international_reputation,physic,shooting,skill_ball_control
0,94.0,95,550000.0,86.0,100500000.0,96.0,5,63.0,89.0,96.0
1,90.0,92,375000.0,81.0,79000000.0,91.0,5,79.0,93.0,92.0
2,89.0,90,275000.0,83.0,54500000.0,92.0,5,64.0,86.0,90.0
3,85.0,90,275000.0,81.0,52500000.0,86.0,5,86.0,91.0,90.0
4,89.0,90,300000.0,NaN,63500000.0,NaN,5,NaN,NaN,31.0


In [16]:
X = players_train_df[selected_features]
y = players_train_df['overall']


In [17]:
nan_counts = players_train_df[selected_features].isnull().sum()

# Print the count of NaN values for each feature
print(nan_counts)

movement_reactions             1
potential                      0
wage_eur                     661
passing                     4968
value_eur                    890
dribbling                   4968
international_reputation       0
physic                      4968
shooting                    4968
skill_ball_control             1
dtype: int64


In [18]:
for feature in selected_features:
    mean_value = players_train_df[feature].mean()
    players_train_df[feature].fillna(mean_value, inplace=True)


In [19]:
X = players_train_df[selected_features]
y = players_train_df['overall']

In [20]:
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

In [21]:
standard_scaler = StandardScaler()
X_standardized = standard_scaler.fit_transform(X_normalized)

In [22]:
X_standardized_df = pd.DataFrame(X_standardized, columns=selected_features)



# Display the standardized DataFrame
print(X_standardized_df.head())

   movement_reactions  potential   wage_eur       passing  value_eur  \
0            3.494261   3.885122  21.821092  2.882230e+00  24.469178   
1            3.054476   3.417121  14.690636  2.383520e+00  19.144450   
2            2.944529   3.105121  10.616090  2.583004e+00  13.076737   
3            2.504744   3.105121  10.616090  2.383520e+00  12.581413   
4            2.944529   3.105121  11.634727 -8.083730e-16  15.305692   

   dribbling  international_reputation        physic      shooting  \
0   3.391069                  9.539965 -2.648456e-01  2.800305e+00   
1   2.893259                  9.539965  1.514662e+00  3.108951e+00   
2   2.992821                  9.539965 -1.536263e-01  2.568820e+00   
3   2.395449                  9.539965  2.293197e+00  2.954628e+00   
4   0.000000                  9.539965  7.902603e-16  3.383827e-16   

   skill_ball_control  
0            2.324515  
1            2.074601  
2            1.949645  
3            1.949645  
4           -1.736575  


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_standardized_df, y, test_size=0.2, random_state=42)

In [24]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
lr_model = LinearRegression()
svr_model = SVR()

In [25]:
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

In [26]:
lr_model.fit(X_train, y_train)
lr_y_pred = lr_model.predict(X_test)

In [27]:
# Train the SVR model
svr_model.fit(X_train, y_train)
svr_y_pred = svr_model.predict(X_test)

In [28]:
def evaluate_model(y_test, y_pred, model_name):
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} - Mean Squared Error: {mse:.2f}")
    print(f"{model_name} - R^2 Score: {r2:.2f}")


In [29]:
evaluate_model(y_test, rf_y_pred, "Random Forest")
evaluate_model(y_test, lr_y_pred, "Linear Regression")
evaluate_model(y_test, svr_y_pred, "SVR")

Random Forest - Mean Squared Error: 1.22
Random Forest - R^2 Score: 0.97
Linear Regression - Mean Squared Error: 7.16
Linear Regression - R^2 Score: 0.85
SVR - Mean Squared Error: 2.92
SVR - R^2 Score: 0.94


In [30]:
estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('lr', LinearRegression()),
    ('svr', SVR())
]


In [31]:
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)


In [32]:
stacking_regressor.fit(X_train, y_train)

# Make predictions with the stacking regressor
stacking_y_pred = stacking_regressor.predict(X_test)


In [33]:
evaluate_model(y_test, stacking_y_pred, "Stacking Regressor")

Stacking Regressor - Mean Squared Error: 1.21
Stacking Regressor - R^2 Score: 0.98


In [ ]:
cv_scores = cross_val_score(stacking_regressor, X_standardized_df, y, cv=5, scoring='r2')
print(f"Stacking Regressor - Cross-Validation R^2 Scores: {cv_scores}")
print(f"Stacking Regressor - Mean Cross-Validation R^2 Score: {np.mean(cv_scores)}")


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the base models
estimators = [
    ('rf', RandomForestRegressor(random_state=42)),
    ('lr', LinearRegression()),
    ('svr', SVR())
]

# Define the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [None, 10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'svr__C': [0.1, 1, 10],
    'svr__gamma': ['scale', 'auto'],
    'svr__kernel': ['rbf', 'linear']
}

# Perform grid search
grid_search = GridSearchCV(stacking_regressor, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_stacking_regressor = grid_search.best_estimator_
print(f"Best parameters for Stacking Regressor: {grid_search.best_params_}")

# Evaluate the best model
stacking_y_pred = best_stacking_regressor.predict(X_test)
evaluate_model(y_test, stacking_y_pred, "Best Stacking Regressor")
